In [2]:
import re
import sys
import time
sys.path.insert(0, './..')

import transformers
from transformers import AutoTokenizer

from utils import load_dataset, calculate_classification_metric, write_classification_metric

In [ ]:
dataset = load_dataset('./../dataset/test.csv')

In [ ]:
model = 'meta-llama/Llama-2-7b-chat-hf'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model, device_map='auto')
model = transformers.AutoModelForCausalLM.from_pretrained(model, device_map='auto')

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': '\
            You will be provided with a string. \
            Distinguish if the string causes SQL injection as a web security specialist.\
            If the string can cause SQL injection, return 1. If not, return 0.\
            Return only result for answer. Do not explain the result.'
    },
    {
        'role': 'user',
        'content': '<string>\' or \'a\' = \'a</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    },
    {
        'role': 'user',
        'content': '<string>Hello.</string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>I am a Japanese<./string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>-3756\' )  union all select 1034,1034,1034--</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    }
]

In [ ]:
classification_counts = {
    'true_positives': 0,
    'false_positives': 0,
    'true_negatives': 0,
    'false_negatives': 0
}
inference_time = []
labels = []
raw_queries = []
preds = []

config = transformers.GenerationConfig(do_sample=True, temperature=0.1, pad_token_id=tokenizer.eos_token_id, max_time=2)
for i, (query, label) in enumerate(dataset[:10]):
    prompt.append(
        {
            'role': 'user',
            'content': f'<string>{query}</string>'
        })
    tokenized_prompt = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, return_tensors="pt")

    start_time = time.time()
    outputs = model.generate(tokenized_prompt, config, max_length=len(tokenized_prompt[0])+2)
    inference_time.append(time.time() - start_time)

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(response[-3:])
    try:
        result = int(re.search(r'(\d+)', response[-3:]).group(1))
    except ValueError:
        result = 'N/A'
    except AttributeError:
        result = 'N/A'

    labels.append(int(label))
    raw_queries.append(query)
    preds.append(result)

    print(i, result, label)
    prompt.pop(9)

calculate_classification_metric(classification_counts, labels, preds, raw_queries)
write_classification_metric(classification_counts, inference_time)